In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from collections import defaultdict
import os
import xml.etree.ElementTree as et
import pandas as pd


Pre Processing


In [4]:
path='/content/drive/My Drive/Ai_project_hmm/Train-corpus'
transition_matrix = defaultdict(dict)
probability_matrix = defaultdict(dict)
initial_matrix = defaultdict(dict)
different_tags = []
different_words = {}
new_word = "#un#"
cardinal_word = "#un1#"

In [5]:
co_trans = defaultdict(dict)
co_tag_word = defaultdict(dict)
co_initial = defaultdict(dict)
p=[.85,.15]

for _dir,subdir,files in os.walk(path):
    for file in files:
            file_path=os.path.join(_dir,file)
            tree=et.parse(file_path)
            root=tree.getroot()
            prev = []
            prev_word='.'
            for word in root.findall('.//'):
                if word.tag=='w' or word.tag=='c' and w!=None:
                    w=word.text
                    tag=word.attrib.get('c5')
                    w=w.lower()
                    w=w.replace(' ','')
                    if w not in different_words:
                        different_words[w] = 1
                    taglist = tag.split('-')
                    if len(taglist)==1 :
                        taglist.append(taglist[0])

                    for i in taglist:
                        if i not in different_tags:
                            different_tags.append(i)
                    
                    co = 0;

                    for j in taglist:
                        if w not in co_tag_word[j] :
                            co_tag_word[j][w]=0
                        co_tag_word[j][w] += p[co]
                        if prev_word == '.':
                            if co_initial[j]=={}:
                                co_initial[j]=0
                            co_initial[j]+=p[co]
                        co+=1
                    if prev!=[]:
                        for i in range(0,2):
                            for j in range(0,2):
                                if taglist[j] not in co_trans[prev[i]]: 
                                    co_trans[prev[i]][taglist[j]]=0
                                co_trans[prev[i]][taglist[j]]+=p[i]*p[j]
                    if w.isdecimal():
                        cardinal_word = w
                    prev = taglist


load = {}
for i in different_tags:
    x = .2
    if i=='NP0' :
        x=.8
    load[i] = x;

for tag in co_tag_word:
    min_key = min(co_tag_word[tag],key = co_tag_word[tag].get)
    co_tag_word[tag][new_word]=load[tag]*co_tag_word[tag][min_key]

load2 = .1;

co_initial_sum = sum(co_initial.values())
for tag in co_initial:
    initial_matrix[tag] = co_initial[tag]/co_initial_sum

for i in co_trans:
    min_key = min(co_trans[i], key = co_trans[i].get)
    for j in different_tags:
        if j not in co_trans[i]:
            co_trans[i][j] = load2*co_trans[i][min_key] 

for i in co_trans:
    co_trans_sum = 0;
    co_trans_sum += sum(co_trans[i].values())
    for j in co_trans[i]:
        if j not in transition_matrix[i]:
             transition_matrix[i][j]=0
        transition_matrix[i][j] += co_trans[i][j]/co_trans_sum

for tag in co_tag_word:
    local_sum = sum(co_tag_word[tag].values())
    for x in co_tag_word[tag]:
        if x not in probability_matrix[tag]:
             probability_matrix[tag][x]=0
        probability_matrix[tag][x] += co_tag_word[tag][x]/local_sum

for tag in different_tags:
    if tag not in initial_matrix:
        initial_matrix[tag]=0

df1 = pd.DataFrame.from_dict(transition_matrix)
df2 = pd.DataFrame.from_dict(probability_matrix)
df3 = pd.DataFrame(initial_matrix, index =[0])

df1.to_csv('/content/drive/My Drive/Ai_project_hmm/transition_matrix')
df2.to_csv('/content/drive/My Drive/Ai_project_hmm/probability_matrix')
df3.to_csv('/content/drive/My Drive/Ai_project_hmm/initial_matrix')


Veterbi Algorithm


In [6]:
test_path = '/content/drive/MyDrive/Ai_project_hmm/Test-corpus'

In [8]:
count = 0;
delta_matrix = {}
back_pointer = defaultdict(dict)
ma_list = []
t = 0
correct_assigned = 0
incorrect_assigned = 0
new_correct_assigned = 0;
new_incorrect_assigned = 0;
for _dir,subdir,files in os.walk(test_path):
    for file in files:
        file_path=os.path.join(_dir,file)
        tree=et.parse(file_path)
        root=tree.getroot()
        for word in root.findall('.//'):
            if (word.tag=='w' or word.tag=='c') and word.text != None:
                w=word.text
                tag=word.attrib.get('c5')
                w=w.lower()
                w=w.replace(' ','')
                if w.isdecimal():
                    w=cardinal_word
                taglist = tag.split('-')
                if w not in different_words:
                    w=new_word
                if len(delta_matrix) == 0:
                    for tag in different_tags:
                        y = 0
                        if w in probability_matrix[tag]:
                            y = probability_matrix[tag][w]
                        delta_matrix[tag]=initial_matrix[tag]*y
                else:
                    new_delta_matrix = {}
                    for tag in different_tags:
                        x = 0
                        back = "NP0"
                        for prev in different_tags:
                            z1 = delta_matrix[prev]
                            z2 = transition_matrix[prev][tag]
                            if x<(z1*z2) :
                                x = max(x, z1*z2)
                                back = prev
                        y = 0
                        if w in probability_matrix[tag]:
                            y=probability_matrix[tag][w]
                        x *= y
                        new_delta_matrix[tag] = x
                        back_pointer[t][tag]=back;
                    delta_matrix=new_delta_matrix
                    t+=1

                count+=1
                del_sum=sum(delta_matrix.values())
                # debug

                #debug
                for i in delta_matrix:
                    delta_matrix[i]/=del_sum
                ma_list.append(taglist)
                if w=='.':
                    assigned_tag = max(delta_matrix, key = delta_matrix.get)
                    assigned_lis = [assigned_tag]
                    cur_tag=assigned_tag
                    while t!=0:
                        t-=1
                        assigned_lis.append(back_pointer[t][cur_tag])
                        cur_tag=assigned_lis[-1]
                    assigned_lis.reverse()
                    ma = len(assigned_lis)
                    for i in range(0,ma):
                        if assigned_lis[i] in ma_list[i]:
                            correct_assigned+=1;
                            # if w == new_word || w == cardinal_word:
                            #     new_correct_assigned+=1;
                        else: 
                            incorrect_assigned+=1
                            # if w == new_word || w == cardinal_word:
                            #     new_incorrect_assigned+=1;
                    t=0;
                    back_pointer = defaultdict(dict)
                    ma_list = []
                    delta_matrix = {}

print("Total accuracy: ")
print(correct_assigned/(incorrect_assigned+correct_assigned))



Total accuracy: 
0.9635654323331393
